In [78]:
import Utils
from DataLoader import DataLoader
from FeatureCreator import FeatureCreator
from Model import Model
from CouponCreator import CouponCreator

config = Utils.read_json('../config.json')
Utils.print_json(config)


dataloader = DataLoader(config)
dataset = dataloader.get_dataset()

{
    "data": {
        "path": "../data/",
        "files": {
            "baskets": "baskets.parquet",
            "coupons": "coupons.parquet",
            "coupons_index": "coupon_index.parquet"
        }
    },
    "model": {
        "test_week": 90,
        "n_shoppers": 2000,
        "n_products": 250,
        "train_window": 4,
        "trend_windows": [
            1,
            3,
            5
        ],
        "n_coupons": 5,
        "discounts": [
            0.15,
            0.2,
            0.25,
            0.3
        ]
    }
}
Read dataset.parquet.gzip from cache...
Successfully read dataset.parquet.gzip into memory.


In [79]:
no_discount = dataset[dataset['discount'] == 0]
no_discount_buy = dataset[dataset['price']  > 0]

discount_30 = dataset[dataset['discount'] == 0.3]
discount_30_buy = dataset[dataset['price'] > 0]


no_discount_all = no_discount.groupby(['week'])['product'].apply(list).reset_index(name='no_discount_all')
no_discount_buy = no_discount.groupby(['week'])['product'].apply(list).reset_index(name='no_discount_buy')
discount_30_all = no_discount.groupby(['week'])['product'].apply(list).reset_index(name='discount_30_all')
discount_30_buy = no_discount.groupby(['week'])['product'].apply(list).reset_index(name='discount_30_buy')

dataset = dataset.merge(no_discount_all, how='left', on=(['week']))
dataset = dataset.merge(no_discount_buy, how='left', on=(['week']))
dataset = dataset.merge(discount_30_all, how='left', on=(['week']))
dataset = dataset.merge(discount_30_buy, how='left', on=(['week']))

In [80]:
dataset

,week,shopper,product,price,discount,purchased,week_hist,elast,redemption_rate,costumer_redemption_rate,discount_buy,no_discount_all,no_discount_buy,discount_30_all,discount_30_buy
0,86,0,0,688.0,0.0,0.0,None,4.200039,0.040919,NaN,NaN,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
1,86,0,1,560.0,0.0,0.0,None,4.637043,0.102400,0.00,1.000000,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
2,86,0,2,773.0,0.0,0.0,None,3.372063,0.037047,0.00,1.000000,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
3,86,0,3,722.0,0.0,0.0,None,10.231161,0.019473,0.00,1.000000,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
4,86,0,4,620.0,0.0,0.0,"[24, 40, 45, 51, 54, 58, 60, 70]",5.286477,0.057530,0.00,0.272727,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2499995,90,1999,245,NaN,0.0,0.0,"[5, 17, 22, 24, 25, 33, 49, 60, 63, 67, 70, 89]",5.080077,0.032053,0.50,0.285714,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
2499996,90,1999,246,NaN,0.0,0.0,None,3.351649,0.044482,0.00,1.000000,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
2499997,90,1999,247,NaN,0.0,0.0,None,16.926842,0.018242,0.00,1.000000,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
2499998,90,1999,248,NaN,0.0,0.0,None,3.703375,0.126265,0.00,1.000000,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."


In [76]:
DATALOADER
    
    '''
    Helper to derive price elasticities
    create list for 
    '''
    
no_discount = dataset[dataset['discount'] == 0]
no_discount_buy = dataset[dataset['price']  > 0]

discount_30 = dataset[dataset['discount'] == 0.3]
discount_30_buy = dataset[dataset['price'] > 0]


no_discount_all = no_discount.groupby(['week'])['product'].apply(list).reset_index(name='no_discount_all')
no_discount_buy = no_discount.groupby(['week'])['product'].apply(list).reset_index(name='no_discount_buy')
discount_30_all = no_discount.groupby(['week'])['product'].apply(list).reset_index(name='discount_30_all')
discount_30_buy = no_discount.groupby(['week'])['product'].apply(list).reset_index(name='discount_30_buy')

dataset = dataset.merge(no_discount_all, how='left', on=(['week']))
dataset = dataset.merge(no_discount_buy, how='left', on=(['week']))
dataset = dataset.merge(discount_30_all, how='left', on=(['week']))
dataset = dataset.merge(discount_30_buy, how='left', on=(['week']))



FEATURECREATOR

for product in dataset['product'].unique()
    no_discount_all = dataset['no_discount_all'][product].count()
    no_discount_buy = dataset['no_discount_buy'][product].count()
    discount_30_all = dataset['discount_30_all'][product].count()
    discount_30_buy = dataset['discount_30_buy'][product].count()
    
    reg_price_buy_rate = no_discount_buy / no_discount_all
    reg_price_offer = total_basket_count - all_discounts_offers

    
    product_elast = (discount_buy_rate - reg_price_buy_rate) / (0.3 * reg_price_buy_rate)

In [ ]:
DATALOADER

def elasticities_prep(dataset):
    '''
    Helper to derive price elasticities
    create list for 
    '''
    no_discount = baskets[baskets['discount'] == 0]
    no_discount_buy = baskets[baskets['price']  > 0]

    discount_30 = baskets[baskets['discount'] == 30]
    discount_30_buy = baskets[baskets['price'] > 0]


    no_discount = no_discount.groupby(['week'])['product'].apply(list).reset_index(name='no_discount')
    no_discount_buy = no_discount.groupby(['week'])['product'].apply(list).reset_index(name='no_discount_buy')
    discount_30 = no_discount.groupby(['week'])['product'].apply(list).reset_index(name='discount_30')
    discount_30_buy = no_discount.groupby(['week'])['product'].apply(list).reset_index(name='discount_30_buy')

    reg_price_offer = 100000 - discount_30


FEATURECREATOR

for product in baskets['products'].unique()
    discount_buy_rate = discount_30_buy / discount_30
    reg_price_buy_rate = no_discount_buy / no_discount
    elast = []
    temp = pd.DataFrame()
    temp['product'] = np.arange(250)
    temp['week'] = i + 1
    elast.append((discount_buy_rate - reg_price_buy_rate) / (0.3 * reg_price_buy_rate))

    
    
    
    
                reg_price_buy = len(reg_price[reg_price['discount'] == 0])
                all_discounts_offers = len(reg_price[reg_price['discount'] > 0])
                reg_price_offer = total_basket_count - all_discounts_offers
                reg_price_buy_rate = reg_price_buy / reg_price_offer
                discount_30 = reg_price[reg_price['discount'] == 30]
                discount_30_offer = len(discount_30)
                discount_30_buy = (discount_30['price'] != 0).sum()
                